attentionの可視化をするコードを追記する予定です

In [1]:
target_model_dir_ = 'runs/Jun03_14-52-17_99986f60c588'

In [2]:
from pathlib import Path

target_model_dir = Path('..', target_model_dir_)
target_model_dir = target_model_dir.resolve()
assert target_model_dir.is_dir(), target_model_dir

In [3]:
import sys

path = str(target_model_dir / 'src')
if path not in sys.path:
    sys.path.insert(0, path)

del sys

In [12]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchvision.transforms.functional import to_pil_image, to_tensor
from torchvision.utils import make_grid
from torchinfo import summary

from IPython.display import display

In [37]:
from model.model import TransformerModel

model = TransformerModel()
model.eval()

summary(model, (2, *model.input_shape), device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
TransformerModel                              --                        --
├─Linear: 1-1                                 [2, 16, 64]               3,200
├─TransformerEncoder: 1-2                     [2, 16, 64]               --
│    └─ModuleList: 2                          --                        --
│    │    └─TransformerEncoderLayer: 3-1      [2, 16, 64]               25,216
│    │    └─TransformerEncoderLayer: 3-2      [2, 16, 64]               25,216
│    │    └─TransformerEncoderLayer: 3-3      [2, 16, 64]               25,216
├─Linear: 1-3                                 [2, 10]                   650
Total params: 79,498
Trainable params: 79,498
Non-trainable params: 0
Total mult-adds (M): 0.06
Input size (MB): 0.01
Forward/backward pass size (MB): 0.21
Params size (MB): 0.32
Estimated Total Size (MB): 0.54

In [38]:
weight_path = sorted(target_model_dir.glob('*.pt'))[-1]
print(weight_path)

model.load_state_dict(torch.load(weight_path))

/workspace/runs/Jun03_14-52-17_99986f60c588/epoch00029.pt


<All keys matched successfully>

In [39]:
transform = transforms.Compose([transforms.ToTensor()])
dataset = MNIST(root='../data', train=False, download=False, transform=transform)

In [45]:
for index in range(2):
    print(index)
    image, label = dataset[index]

    display(to_pil_image(image))

    patches = model.unfold_patch(image[None])[0]
    display(to_pil_image(make_grid(patches, nrow=model.num_patches, pad_value=0.5)))

    output_prob = model(image[None])[0].softmax(0)

    print(output_prob.tolist())

    print(output_prob.argmax())

    print()

0


[0.004493934102356434, 0.05076932907104492, 0.018564721569418907, 0.002767853671684861, 0.01326053962111473, 0.014602843672037125, 0.00039116654079407454, 0.799828827381134, 0.0628809854388237, 0.03243980184197426]
tensor(7)

1


[0.0003057601279579103, 0.004064821172505617, 0.3416615426540375, 0.03231873735785484, 0.00042588802170939744, 0.503757119178772, 0.0001153868215624243, 0.11243999749422073, 0.0040403869934380054, 0.0008704166393727064]
tensor(5)



In [64]:
x = image[None]
x = model.unfold_patch(x)
x = x.flatten(2)
x = model.patch_embedding(x)
x = model.transformer_encoder(x)
x = x.mean(1)
x = model.mlp_head(x)
x.shape, x[0].softmax(0).tolist()

(torch.Size([1, 10]),
 [0.0003057601279579103,
  0.004064821172505617,
  0.3416615426540375,
  0.03231873735785484,
  0.00042588802170939744,
  0.503757119178772,
  0.0001153868215624243,
  0.11243999749422073,
  0.0040403869934380054,
  0.0008704166393727064])

In [59]:
list(model.transformer_encoder.children())[0]

ModuleList(
  (0): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
    )
    (linear1): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear2): Linear(in_features=64, out_features=64, bias=True)
    (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
  )
  (1): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
    )
    (linear1): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear2): Linear(in_features=64, out_features=64, bias=True)
    (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((64,), eps=

In [48]:
model

TransformerModel(
  (patch_embedding): Linear(in_features=49, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.5, inplace=False)
        (dropout2): Dropout(p=0.5, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=